Import Libraries

In [1]:
import sqlite3
import requests
from tqdm import tqdm

from flask import Flask, request
import json 
import numpy as np
import pandas as pd

# Introduction
In most cases when we are running a business, there are a lot of data stakeholder outside our company. The problem is that we need to provide the access in a way that they will not break our security rules or concerns. One way to solve that is by creating an API for the database. In this project, we will introduce you on how python is used for data transaction management using Flask API. 

**Usecase**: Bikeshare App\
Have you ever rent a bike for faster mobility in town? In the past few years, this business once become a phenomenom. In Indonesia, there are lots of similar services, for example, the Jakarta government's "GOWES" bike sharing service that launcehd in July 2020. 

For the user perspective, the general journey is denoted as follows:
- User scan the bike located at some bike station, sending the data to database as the intent of "start riding"
- Once user has reached its destination station, he/she put back the bike, sending the data again to the database as the intent of "finished riding"

For each activity, there are data transactions between user and database. And how do you think each user's phone communicate with the server for storing and receiving the data? Using API ofcourse! 

We will later create a simplified version of the API service which handles data transactions and analysis. 


**Goals**: Make an API service to connect 3rd party user with data using HTTP request

**End Product**: A Flask API which capable of doing: 
- Input new data to database
- Read specific data from database
- Get specific insight from data analysis process (ie: best performing stations)

**Scoring Metrics**: 

1. 1 point - Created Flask App
2. 2 poitns - Created functionality to read or get specific data from the databse
3. 4 points - Created functionality to input new data into each table for the databases
4. 3 points - Created static endpoints which return analytical result (must be different from point 2,3)
5. 3 points - Created dynamic endpoints which return analytical result (must be different from point 2,3,4)
6. 3 points - Created POST endpoint which receive input data, then utilize it to get analytical result (must be different from point 2,3,4,5)

**Tools**: 
- **Python** with **Jupyter Notebook**, installed with **required libraries**
- **Visual Studio Code (VSCode)**: Recommended for writing application scripts
- **TablePlus**: Recommended for easy database access and exploration
- Postman: Optional for easy API testing


# About the Data

The data that we will be using in this project is **[Austin Bike Share]()** dataset which contains information on bike trip start location, stop location, duration, type of bike share user in the city of Austin, Texas. Bike station location data is also provided. 

All the information is stored in a database called **austin_bikeshare.db**. However, we also provides the non existing data in csv files. These data then will be imported into the database using the API

Lists of files:
- **austin_bikeshare.db**: The database, contains `trips` and `stations` table
- **data/austin_bikeshare_stations.csv**: contains all the stations information which is not yet available in the database table
- **data/austin_bikeshare_trips_2021.csv**: contains all 2021 data which is not yet available in the database table

In [2]:
# Reading the csv data
trips = pd.read_csv('data/austin_bikeshare_trips_2021.csv')
stations = pd.read_csv('data/austin_bikeshare_stations.csv')

In [3]:
trips

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,23455589,Local365,174,2021-01-26 17:47:42 UTC,4059.0,Nash Hernandez/East @ RBJ South,3660.0,East 6th/Medina,7
1,23459960,Local365,19265,2021-01-28 08:03:52 UTC,4054.0,Rosewood/Chicon,4055.0,11th/Salina,16
2,23436548,Local365,18331,2021-01-21 07:09:48 UTC,4055.0,11th/Salina,4055.0,11th/Salina,25
3,23382542,Local365,19682,2021-01-04 09:38:18 UTC,4062.0,Lakeshore/Pleasant Valley,3293.0,East 2nd/Pedernales,5
4,23374882,Local365,19437,2021-01-02 15:04:01 UTC,3293.0,East 2nd/Pedernales,4062.0,Lakeshore/Pleasant Valley,5
...,...,...,...,...,...,...,...,...,...
17977,23515345,Local365,19476,2021-02-13 14:17:15 UTC,3838.0,26th/Nueces,3841.0,23rd/Rio Grande,62
17978,23553979,Local365,19418,2021-02-26 18:12:13 UTC,3838.0,26th/Nueces,3841.0,23rd/Rio Grande,4
17979,23499655,U.T. Student Membership,21553,2021-02-07 17:15:29 UTC,3838.0,26th/Nueces,3841.0,23rd/Rio Grande,3
17980,23538444,U.T. Student Membership,2147,2021-02-22 20:52:06 UTC,3838.0,26th/Nueces,3841.0,23rd/Rio Grande,704


## Taking a look for trips data
`trips` table in database, or `austin_bikeshare_trips.csv` in original files. It roughly consisted of 1.3 million rows

In [3]:
trips.head(2)

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,23455589,Local365,174,2021-01-26 17:47:42 UTC,4059.0,Nash Hernandez/East @ RBJ South,3660.0,East 6th/Medina,7
1,23459960,Local365,19265,2021-01-28 08:03:52 UTC,4054.0,Rosewood/Chicon,4055.0,11th/Salina,16


**Data Descriptions**:
- `bikeid`: integer id of bike
- `checkout_time`: HH:MM:SS, see start time for date stamp
- `duration_minutes`: int minutes of trip duration
- `endstationid`: integer id of end station
- `endstationname`: string of end station name
- `month`: month, integer
- `startstationid`: integer id of start station
- `startstationname`: string of start station name
- `start_time`: YYYY-MM-DD HH:MM:SS
- `subscriber_type`: membership typ e.g. walk up, annual, other bike share, etc
- `trip_id`: unique trip id int
- `year`: year of trip, int

## Taking a look for stations data
`stations` table in database, or `austin_bikeshare_stations.csv` in original files 

In [4]:
stations.tail(2)

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
96,3795,Dean Keeton & Whitis,active,300 W. Dean Keeton St.,NaN,32671.0,undetermined_parking,19.0,solar,55.0,5.0,NaN,9,2021-01-04T12:00:00Z
97,3797,21st & University,active,2007 University Ave.,NaN,32674.0,undetermined_parking,19.0,solar,55.0,5.0,ROW might belong to UT,9,2021-01-04T12:00:00Z


**stations table (or austin_bikeshare_trips.csv)**:
- `station_id`: integer id of station
- `name`: string of station name
- `status`: string of station status (active, closed, moved, ACL-only)
- `address`: string of station address
- `alternate_name`: string of station alternative name
- `city_asset_number`: integer of station's asset number
- `property_type`: string of station's property type
- `number_of_docks`: integer of number of available bike docks
- `power_type`: string of station's power source type
- `footprint_length`: float of station' blueprint length (size of the station). Probably in meters
- `footprint_width`: float of station' blueprint width (size of the station). Probably in meters
- `notes`: string of additional notes
- `council_district`: integer of stations's council district
- `modified_date`: date of last modified information regarding the station

# Database

The first important task is to make sure we can securely make connections to the database. In this scenario, we will connect to sqlite database, `austin_bikeshare.db` via python. In this part, we will re-visit on how to work with databases, started with making a connection, get some data, and insert data into it. 

You can directly connect and view the database using TablePlus, or run the following code to create the connection

In [4]:
# Define a function to create connection for reusability purpose
def make_connection():
    connection = sqlite3.connect('austin_bikeshare.db')
    return connection

# Make a connection
conn = make_connection()

In [5]:
conn

## POST: Insert data into database

Generally, POST method will utilize the data sent by user for specific purporse, for example: 

- Insert new data into the database. 
- Operate the data into some function

We will learn how to insert data into a specific table in our database. Please refer to the following code to create and run the query for the given task

In [10]:
# Get the data values
data = tuple(stations.iloc[0].fillna('').values)

# Make the query
query = f"""
INSERT INTO stations
VALUES {data}
"""

In [8]:
# See the actual query looks like
print(query)


INSERT INTO stations
VALUES (2541, 'State Capitol @ 14th & Colorado', 'closed', '206 W. 14th St.', '', '', '', '', '', '', '', '', 1, '2021-01-04T12:00:00Z')



In [11]:
# Execute the query 
conn.execute(query)

In [12]:
# Commit changes to database
conn.commit()

Once the code above succesful, it's recomended to wrap it into a function so that we can reuse it in the future. Complete the following codes to make the function:

In [7]:
def insert_into_stations(data, conn):
    query = f"""INSERT INTO stations values {data}"""
    try:
        conn.execute(query)
    except:
        return 'Error'
    conn.commit()
    return 'OK'

In [10]:
# Example use of the function 

data = tuple(stations.iloc[29].fillna('').values) # Randomly select a data
result = insert_into_stations(data, conn)

In [11]:
result

'OK'

In [42]:
stations.dtypes


station_id             int64
name                  object
status                object
address               object
alternate_name        object
city_asset_number    float64
property_type         object
number_of_docks      float64
power_type            object
footprint_length     float64
footprint_width      float64
notes                 object
council_district       int64
modified_date         object
dtype: object

**TASK: Create a Function to insert data for `trips` table**

In [4]:
# Your code here
def insert_into_trips(data, conn):
    query = f"""INSERT INTO trips values {data}"""
    try:
        conn.execute(query)
    except:
        return 'Error'
    conn.commit()
    return 'OK'


In [5]:
data = tuple(trips.iloc[39].fillna('').values) # Randomly select a data
result1 = insert_into_trips(data, conn)

In [6]:
result1

'OK'

## GET: Read specific data

Generally, GET method will ask for specific the data in the database, alongside with additional information we might send. For example: 
- Get number of unique user
- Get full trips information for specific user id
- Get average trips duration and length for specific user id

In this example, we will implement the most basic GET method, that is to get Station information in the table based on specific Station ID

In [129]:
# Get Specific Station ID  Query
station_id = 2538
query_station_id = f"""
SELECT * FROM stations
WHERE station_id = {station_id}
"""

# Get all station ID Query
query_station_all = "SELECT * FROM stations"

In [130]:
print(query_station_id)


SELECT * FROM stations
WHERE station_id = 2538



In [131]:
# Run the query and get the result
result2 = pd.read_sql_query(query_station_id, conn)

In [132]:
result2

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2538,Bullock Museum @ Congress & MLK,closed,1881 Congress Ave.,,,,,,,,,1,2021-01-04T12:00:00Z


Just like the previous section, create a function to read specific station

In [17]:
station_id = 2541

In [18]:

def get_station_id(station_id, conn):
    query = f"""SELECT * FROM stations WHERE station_id = {station_id}"""
    result = pd.read_sql_query(query, conn)
    return result 

def get_all_stations(conn):
    query = f"""SELECT * FROM stations"""
    result = pd.read_sql_query(query, conn)
    return result

**TASK: Create a Function to read data from `trips` table**

In [20]:
get_station_id(2541, conn)

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2541,State Capitol @ 14th & Colorado,closed,206 W. 14th St.,,,,,,,,,1,2021-01-04T12:00:00Z


In [30]:
# Your code here
def get_trip_id(trip_id, conn):
    query = f"""SELECT * FROM trips WHERE id = {trip_id}"""
    result = pd.read_sql_query(query, conn)
    return result
    
def get_all_trips(conn):
    query = f"""SELECT * FROM trips"""
    result = pd.read_sql_query(query, conn)
    return result

In [32]:
get_trip_id(9900289692, conn)

,id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900289692,Walk Up,248,2015-10-02 21:12:01 UTC,1006,Zilker Park West,1008,Nueces @ 3rd,39


In [33]:
get_all_trips(conn).head(1)

,id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900289692,Walk Up,248,2015-10-02 21:12:01 UTC,1006,Zilker Park West,1008,Nueces @ 3rd,39


# Flask App

Flask is a micro-framework for python. Generally we can build any application out of it. To start with, let's make our first empy flask app. 
Creata a new file `app.py`, then copy-paste the following codes into it and save it. 

*Notes: it's recomended to use vscode as editor since it support vast linting supports, including python which we find really helpful in coding*

```python
from flask import Flask, request
app = Flask(__name__) 

if __name__ == '__main__':
    app.run(debug=True, port=5000)
```

To run the app, you can open your terminal, go to the specific folder and run `python app.py` using your designated virtual environment

## Routes and Endpoints

### Implement endpoints

If you succesfully run the previous app and access the localhost:5000 , you might get an 404 not found error. This is because we **have not** define yet what will happen if people accessing our root path of the app (localhost:5000/)

Add the following example of route or endpoint into your `app.py` just before the `if __name__ === '__main__':` line, and reload the page to see if it works

```python 
@app.route('/')
def home():
    return 'Hello World'
```

Above endpoints will runs the `home` function anytime user access the `/` page.

Since we are going to handle all the requests through this app, the very next step is to implement our functionalites. Let's start with implementing the functionality to read all station data

we will add the `get_all_stations()` functions into our app, create a `@app.route('/stations/')` endpoint and call the function in it. The code should looks like:

```python 
@app.route('/stations/')
def route_all_stations():
    conn = make_connection()
    stations = get_all_stations(conn)
    return stations.to_json()
    
def get_all_stations(conn):
    query = f"""SELECT * FROM stations"""
    result = pd.read_sql_query(query, conn)
    return result
```

However, adding just above codes is not enough. If you see, there is a dependency inside the function, which is `make_connection()` function. Other than that, we will be using all the required libraries. 

Hence, we can complete the code by : 
- import the required libraries at the top of the `app.py` file
- write the `make_connection()` function before any routes declarations

Once it's completed, you should see no errors in the vscode screen, and your endpoint should work just fine and returns all the stations. It's now your task to impelement the `get_all_trips` into the app 

**TASK: Implement `get_all_trips()` method into the app** \
Place it under `@app.route('/trips/')` endpoint

In [36]:
# your code

def get_all_trips(conn):
    query = f"""SELECT * FROM trips"""
    result = pd.read_sql_query(query, conn)
    return result

In [35]:
get_all_trips(conn).head(1)

,id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900289692,Walk Up,248,2015-10-02 21:12:01 UTC,1006,Zilker Park West,1008,Nueces @ 3rd,39


### Access Endpoints

To access our endpoints using python, simply use `requests` library followed by the request method (post, get, put, delete, etc) the passed in the url or data 

In [4]:
url = 'http://127.0.0.1:5000/json/'
res = requests.get(url)

In [5]:
res

<Response [200]>

In [6]:
res.text

'{"station_id":{"0":2541,"1":3464,"2":2538,"3":3687,"4":2575,"5":2498,"6":2574,"7":2496},"name":{"0":"State Capitol @ 14th & Colorado","1":"Pease Park","2":"Bullock Museum @ Congress & MLK","3":"Boardwalk West","4":"Riverside @ S. Lamar","5":"Convention Center \\/ 4th St. @ MetroRail","6":"Zilker Park","7":"8th & Congress"},"status":{"0":"closed","1":"closed","2":"closed","3":"active","4":"active","5":"active","6":"active","7":"active"},"address":{"0":"206 W. 14th St.","1":"1155 Kingsbury St","2":"1881 Congress Ave.","3":"300 E. Riverside Dr.","4":"Presented by Austin Parks Foundation","5":"499 E. 4th St","6":"Barton Springs Rd","7":"111 E. 8th St."},"alternate_name":{"0":"","1":"","2":"","3":"","4":"","5":"","6":"Zilker Park at Barton Springs and William Burton Drive","7":""},"city_asset_number":{"0":"","1":"","2":"","3":16683,"4":16736,"5":16734,"6":16759,"7":16725},"property_type":{"0":"","1":"","2":"","3":"parkland","4":"parkland","5":"sidewalk","6":"parkland","7":"sidewalk"},"numb

The response sent by the server was not only contains bare data. It was packed as HTTP response, meaning that we need to unpack the response first  in order to get the data. we can use `res.json()` to obtain it, then use pandas to transform it into dataframe for readibility or future analysis purposes. 

In [ ]:
pd.DataFrame(res.json())#.head()

## Static and Dynamic Endpoints

On previous part we already made several endpoints which control how our server will react whenever users access it. 
If we take a look at the endpoints, it's all static : 
- `@app.route('/')`
- `@app.route('/home/')`
- `@app.route('/stations/')`
- `@app.route('/trips/')`


What if, instead of getting all the stations information, we only wanted to read a specific station information of station_id 3464 ? \
should we make an excact endpoint of that like `@app.route('stations/3464')`?. But what about the others? are we going to make a bunch of endpoints for each specific id like: 
- `@app.route('/stations/3464')`
- `@app.route('/stations/2500')`
- `@app.route('/stations/2541')`
- and so on...

Of course we shouldn't. One way to overcome the problem is by delcaring a **dynamic endpoints**. Basically, it's an endpoint which allows user to insert a variable values in it. The final looks of the dynamic endpoint version of above problem is: 
- `@app.route('stations/<station_id>')`

From above endpoint, the \<station_id\> is the variable, and we need to make sure that it influence how our following function acts. The whole dynamic endpoints for getting specific station by its id will looks like: 

```python 
@app.route('/stations/<station_id>')
def route_stations_id(station_id):
    conn = make_connection()
    station = get_station_id(station_id, conn)
    return station.to_json()
```


Before accesing the endpoint, make sure that `make_connection()` and `get_station_id()` functions are included inside `app.py` script so that it won't raise an error. Now for the final step, we can try to access the dynamic endpoints by changing the `station_id` in the following codes

In [37]:
station_id = 3464
url = f"http://127.0.0.1:5000/stations/{station_id}"

In [38]:
response = requests.get(url)
pd.DataFrame(response.json())

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,3464,Pease Park,closed,1155 Kingsbury St,,,,,,,,,9,2021-01-04T12:00:00Z


In [39]:
url

'http://127.0.0.1:5000/stations/3464'

**TASK: Implement dynamic endpoints to read specific trip by its trip_id!** \
Place it under `@app.route('/trips/<trip_id>')` endpoint

## Handling JSON data as input

Sometimes, in order to make something happens in our API, we need the user to send us the data. In this case, we need to handle how we can get the data (which mostly sent as json format) and utilize it inside our endpoint functions. 

In order to achieve that, we will be using flask's `request` classes.


```python
@app.route('/json') 
def json_example():
    
    req = request.get_json(force=True) # Parse the incoming json data as Dictionary
    
    name = req['name']
    age = req['age']
    address = req['address']
    
    return (f'''Hello {name}, your age is {age}, and your address in {address}
            ''')
```

In [31]:
data = {
    "name" : "Andi", 
    "age" : 24 ,
    "address" : "my adress"
}

url = "http://127.0.0.1:5000/json"

In [32]:
res = requests.post(url, json=data)

In [21]:
res

<Response [200]>

In [33]:
res.text

'Hello Andi, your age is 24, and your adress is in my adress\n            '

Now that we already know how to handle json input, we can try to implement and endpoint in which we can insert new data into `stations` table. 

```python

@app.route('/stations/add', methods=['POST']) 
def route_add_station():
    # parse and transform incoming data into a tuple as we need 
    data = pd.Series(eval(request.get_json(force=True)))
    data = tuple(data.fillna('').values)
    
    conn = make_connection()
    result = insert_into_stations(data, conn)
    return result


```

In [6]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [19]:
data = stations.iloc[24].fillna('').to_dict()
data_json = json.dumps(data, cls=NpEncoder)

In [20]:
data_json

'{"station_id": 2575, "name": "Riverside @ S. Lamar", "status": "active", "address": "Presented by Austin Parks Foundation", "alternate_name": "", "city_asset_number": 16736.0, "property_type": "parkland", "number_of_docks": 15.0, "power_type": "solar", "footprint_length": 45.0, "footprint_width": 5.0, "notes": "parkland at ROW/easement", "council_district": 5, "modified_date": "2021-01-04T12:00:00Z"}'

In [21]:
url = "http://127.0.0.1:5000/stations/add"
res = requests.post(url, json=data_json)

In [22]:
res

<Response [200]>

In [23]:
res.text

'Error'

In [24]:
# Sintaks for iteratively insert stations (from stations_csv) into stations table

for idx, data in tqdm(stations.iterrows()):
    data = data.fillna('').to_dict()
    data_json = json.dumps(data, cls=NpEncoder)
    
    url = "http://127.0.0.1:5000/stations/add"
    res = requests.post(url, json=data_json)

98it [00:00, 178.38it/s]


**TASK: Using the API, insert all the trips data (in csv) into the database by running the following codes**\
It might run differently on each devices, so you might get some rest while waiting for it to complete

In [16]:
trips.shape

(17982, 9)

In [17]:
stations.shape

(98, 14)

In [28]:
for idx, data in tqdm(trips.iloc[15001 : 17982].iterrows()):
    data = data.fillna('').to_dict()
    data_json = json.dumps(data, cls=NpEncoder)
    
    url = "http://127.0.0.1:5000/trips/attach"
    res = requests.post(url, json=data_json)
    
#iter bertahap, prone to error

2981it [01:09, 43.15it/s]


In [ ]:
res

# [Gema] List of endpoints: 

- http://127.0.0.1:5000/trips

- http://127.0.0.1:5000/stations

- http://127.0.0.1:5000/trips/attach (trips/add) -> post endpoint (example)

- http://127.0.0.1:5000/stations/add -> post endpoint (example)

- http://127.0.0.1:5000/trips/rent -> post endpoint

- http://127.0.0.1:5000/trips/goaround/ -> static + dynamic endpoint, can be subsetted by 'id's from 'trips'table; 
  to seek insight of trips which is start & ended at the same station; *triggering promos, bundling packages associated by its venue/place*
  
- http://127.0.0.1:5000/trips/average_duration -> static endpoint only
  showing the avg value of all durations from 'trips'


# Make your own analytic endpoints

Based on what we've learned before, create your own endpoints which returns analytical result such as contigency tables, aggregation tables, or even just a values.

*ps: if the return is dataframe/series, don't forget to change it into json with `.to_json()` method*


- [x] 1 point - Created Flask App
- [x] 4 points - Created functionality to input new data into each table for the databases
- [x] 2 poitns - Created functionality to read or get specific data from the database
- [ ] 3 points - Created static endpoints which return analytical result (must be different from point 2,3)
- [ ] 3 points - Created dynamic endpoints which return analytical result (must be different from point 2,3,4)
- [ ] 3 points - Created POST endpoint which receive input data, then utilize it to get analytical result (must be different from point 2,3,4,5)


## Create Static Endpoint(s)

You can use the following cell to try-out your function before implementing it as an endpoint

In [ ]:
@app.route('/stations/add', methods=['POST']) 
def route_add_station():
    # parse and transform incoming data into a tuple as we need 
    data = pd.Series(eval(request.get_json(force=True)))
    data = tuple(data.fillna('').values)

    conn = make_connection()
    result = insert_into_stations(data, conn)
    return result

In [37]:
def get_average_duration(conn):
    query = f"""SELECT id, AVG (duration_minutes) as  AVG_duration"""
    result = pd.read_sql_query(query, conn)
    return result

@app.route('/trips/average_duration') 
def route_average_duration():
    conn = make_connection()
    avg_dur = get_average_duration(conn)
    return avg_dur.to_json()
    
    


In [ ]:
@app.route('/trips/goaround') 
def route_goaround_trips():
    conn = make_connection()
    go_arnd = get_goaround_trips(conn)
    return go_arnd.to_json()

In [38]:
def get_goaround_trips(conn):
    query = f"""SELECT * from Trips, WHERE trips.start_station_id like trips.end_station_id"""
    result = pd.read_sql_query(query, conn)
    return result

## Create Dynamic Endpoints 

You can use the following cell to try-out your function before implementing it as an endpoint

In [43]:
def get_goaround_id(id, conn):                                         #station dynamic route's func#
    query = f"""SELECT * FROM trips WHERE id = {id}"""   
    result = pd.read_sql_query(query, conn)
    return result 

In [ ]:
@app.route('/trips/goaround/<id>')                                          
def route_goaround_id(id): 
    conn = make_connection()
    go_arnd_id = get_goaround_trips(id, conn)
    return go_arnd_id.to_json()

## Create POST Endpoints

You can use the following cell to try-out your function before implementing it as an endpoint.

If you still find it quite difficult, here's an example case you might wanted to try on:
- input : a dictionary contained a datetime period
    - { "period" : "2015-08" }
- output:
    - Aggregation table of bike rent activities for each station in that specific period
- example code: 

```python 
input_data = request.get_json() # Get the input as dictionary
specified_date = input_data['period'] # Select specific items (period) from the dictionary (the value will be "2015-08")

# Subset the data with query 
conn = make_connection()
query = f"SELECT * FROM stations WHERE start_time LIKE ({specified_date}%)"
selected_data = pd.read_sql_query(query, conn)

# Make the aggregate
result = selected_data.groupby('start_station_id').agg({
    'bikeid' : 'count', 
    'duration_minutes' : 'mean'
})

# Return the result
return result.to_json()

```

In [9]:
specified_date = {
    'x' : "2015-02", 
}

url = "http://127.0.0.1:5000/trips/rent"
res = requests.post(url, json=specified_date)

In [11]:
res.text

'{"bikeid":{"1008":6,"2494":411,"2495":386,"2496":233,"2497":350,"2498":247,"2499":462,"2501":423,"2502":261,"2503":251,"2504":196,"2536":150,"2537":175,"2538":213,"2539":161,"2540":125,"2541":99,"2542":154,"2544":104,"2545":52,"2547":126,"2548":197,"2549":278,"2550":277,"2552":158,"2561":73,"2562":147,"2563":324,"2564":83,"2565":193,"2566":290,"2567":173,"2568":81,"2569":145,"2570":248,"2571":105,"2572":156,"2575":431,"2707":400,"2711":250,"2712":137,"2822":153,"2823":59,"":629},"duration_minutes":{"1008":5.5,"2494":28.3260340633,"2495":29.707253886,"2496":14.6394849785,"2497":45.0942857143,"2498":19.1295546559,"2499":32.6212121212,"2501":15.7423167849,"2502":31.183908046,"2503":35.5418326693,"2504":25.8316326531,"2536":37.3533333333,"2537":21.5942857143,"2538":22.544600939,"2539":24.8322981366,"2540":24.52,"2541":31.303030303,"2542":17.6298701299,"2544":24.7980769231,"2545":22.9230769231,"2547":35.7777777778,"2548":30.4010152284,"2549":33.6223021583,"2550":22.9061371841,"2552":16.379